In [4]:
import numpy as np
import pandas as pd
import csv, sqlite3

In [5]:
conn = sqlite3.connect(":memory:") # change to 'sqlite:///your_filename.db'

In [15]:
activity_log_df = pd.read_csv("ACTIVITY_LOG.csv")
component_df = pd.read_csv("COMPONENT_CODES.csv")
user_log_df = pd.read_csv("USER_LOG.csv")

In [16]:
col_name_change={'User Full Name *Anonymized':'Uid'}
activity_log_df.rename(columns=col_name_change, inplace=True)
user_log_df.rename(columns=col_name_change, inplace=True)

In [23]:
m_df = pd.merge(user_log_df,activity_log_df,left_index=True, right_index=True, suffixes=('_user', '_activity'))
m_df

,Date,Time,Uid_user,Uid_activity,Component,Action,Target
0,17/11/2023 00:00,00:00:00,129,129,Course,Viewed,Content
1,10/11/2023 00:00,00:00:01,26,26,Quiz,Updated,Response
2,10/11/2023 00:00,00:00:02,26,26,Quiz,Viewed,Attempt
3,20/12/2023 00:00,00:00:02,86,86,Assignment,Viewed,Assignment
4,20/12/2023 00:00,00:00:03,86,86,Assignment,Viewed,Submission_state
...,...,...,...,...,...,...,...
150830,20/11/2023 00:00,23:59:47,125,125,Course,Viewed,Content
150831,19/12/2023 00:00,23:59:47,86,86,System,Viewed,Content
150832,12/10/2023 00:00,23:59:50,129,129,Course,Viewed,Course
150833,09/11/2023 00:00,23:59:56,26,26,Quiz,Viewed,Attempt


In [24]:
m_df.Uid_user == m_df.Uid_activity

0         True
1         True
2         True
3         True
4         True
          ... 
150830    True
150831    True
150832    True
150833    True
150834    True
Length: 150835, dtype: bool

In [25]:
m_df.Uid_user != m_df.Uid_activity

0         False
1         False
2         False
3         False
4         False
          ...  
150830    False
150831    False
150832    False
150833    False
150834    False
Length: 150835, dtype: bool

In [29]:
differences = m_df.Uid_user[m_df.Uid_user != m_df.Uid_activity]
differences

Series([], Name: Uid_user, dtype: int64)

In [9]:
user_log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150835 entries, 0 to 150834
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Date    150835 non-null  object
 1   Time    150835 non-null  object
 2   Uid     150835 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.5+ MB


In [6]:
merge_log_df = pd.concat([user_log_df, activity_log_df.drop('Uid', axis=1)], axis=1)
merge_log_df

,Date,Time,Uid,Component,Action,Target
0,17/11/2023 00:00,00:00:00,129,Course,Viewed,Content
1,10/11/2023 00:00,00:00:01,26,Quiz,Updated,Response
2,10/11/2023 00:00,00:00:02,26,Quiz,Viewed,Attempt
3,20/12/2023 00:00,00:00:02,86,Assignment,Viewed,Assignment
4,20/12/2023 00:00,00:00:03,86,Assignment,Viewed,Submission_state
...,...,...,...,...,...,...
150830,20/11/2023 00:00,23:59:47,125,Course,Viewed,Content
150831,19/12/2023 00:00,23:59:47,86,System,Viewed,Content
150832,12/10/2023 00:00,23:59:50,129,Course,Viewed,Course
150833,09/11/2023 00:00,23:59:56,26,Quiz,Viewed,Attempt


In [7]:
component_df.to_sql("components", conn, if_exists='append', index=False)
merge_log_df.to_sql("merge_log", conn, if_exists='append', index=False)

150835

In [11]:
cur = conn.cursor()
for row in cur.execute('SELECT * FROM components;'):
    print(row)

('Course', 'Cour')
('Quiz', 'Quiz')
('Assignment', 'Assign')
('System', 'Sys')
('Lecture', 'Lect')
('Study_material', 'Study_mat')
('Manual', 'Man')
('Survey', 'Survey')
('Folder', 'Fold')
('Attendence', 'Attend')
('Page', 'Page')
('Test', 'Test')
('URL', 'Url')
('Book', 'Book')
('Source', 'Sourc')
('Questionnaire', 'Quest')
('Feedback', 'Feedback')
('Project', 'Proj')
